In [15]:
import dask
import dask_image.imread
from dask.diagnostics import ProgressBar

from PIL import Image
import glob
import numpy as np

In [2]:
all_imgs = dask_image.imread.imread('./chest_xray/train/**/*.jpeg')

In [3]:
all_imgs

,Array,Chunk
Bytes,20.25 GB,3.88 MB
Shape,"(5216, 1858, 2090)","(1, 1858, 2090)"
Count,15648 Tasks,5216 Chunks
Type,uint8,numpy.ndarray


In [4]:
with ProgressBar():
    x = all_imgs.mean().compute(scheduler='processes', num_workers=8)

[                                        ] | 0% Completed |  0.6s


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 4 dimension(s)

In [6]:
all_imgs[0].compute().shape

(1858, 2090)

In [5]:
all_imgs[1341].compute().shape

(439, 712, 3)

In [17]:
@dask.delayed
def open_image_rgb(fn):
    img = Image.open(fn).convert(mode='RGB')
    return np.array(img)

In [9]:
files = glob.glob('./chest_xray/train/**/*.jpeg')

In [21]:
lazy_images = [open_image_rgb(fn) for fn in files]
sample = lazy_images[0].compute()

arrays = [dask.array.from_delayed(lazy_image,
                          dtype=sample.dtype,
                          shape=sample.shape)
          for lazy_image in lazy_images]

all_imgs = dask.array.stack(arrays, axis=0)

In [22]:
all_imgs

,Array,Chunk
Bytes,18.59 GB,3.56 MB
Shape,"(5216, 1024, 1160, 3)","(1, 1024, 1160, 3)"
Count,15648 Tasks,5216 Chunks
Type,uint8,numpy.ndarray


In [25]:
all_imgs[0].mean(axis=(0, 1)).compute()

array([127.09697097, 127.09697097, 127.09697097])

In [29]:
with ProgressBar():
    pixel_mean = all_imgs.mean(axis=(0, 1, 2)).compute(scheduler='processes', num_workers=8)
    
pixel_mean

[########################################] | 100% Completed | 56.4s


array([124.30439269, 124.30439269, 124.30439269])

In [30]:
with ProgressBar():
    pixel_std = all_imgs.std(axis=(0, 1, 2)).compute(scheduler='processes', num_workers=8)
    
pixel_std

[########################################] | 100% Completed |  1min 29.4s


array([62.62273996, 62.62273996, 62.62273996])

In [31]:
pixel_mean / 255.0, pixel_std / 255.0

(array([0.48746821, 0.48746821, 0.48746821]),
 array([0.24557937, 0.24557937, 0.24557937]))